In [6]:
import cv2
import threading
import playsound
import smtplib
import tkinter as tk
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage

fire_cascade = cv2.CascadeClassifier('fire_detection_cascade_model.xml')
vid = cv2.VideoCapture(0)
runOnce = False

def play_alarm_sound_function():
    playsound.playsound('fire_alarm.mp3', True)
    print("Fire alarm end")

def send_mail_function(image):
    recipientmail = "bagdekunal96@gmail.com"
    recipientmail = recipientmail.lower()

    # Put sender email here
    sender_mail = "bagdekunal2804@gmail.com"
    password = "udiivtlgvsywzaks"
    
    try:
        # Create message container
        msg = MIMEMultipart()
        msg['From'] = sender_mail
        msg['To'] = recipientmail
        msg['Subject'] = 'Fire detected'
        
        # Add message body
        body = '''Warning: Fire accident has been reported
                Address: Rajiv Gandhi Institute Of Technology
                Juhu Versova Link Rd,
                behind HDFC Bank, Gharkul Society,
                Bharat Nagar, Versova, Andheri West,
                Mumbai, Maharashtra 400053 '''
        msg.attach(MIMEText(body, 'plain'))


        # Attach the image to the email
        img_attach = MIMEImage(image, name='fire.jpg')
        msg.attach(img_attach)

        # Create SMTP session
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_mail, password)

        # Send mail
        text = msg.as_string()
        server.sendmail(sender_mail, recipientmail, text)

        # Terminate SMTP session
        server.quit()
        print("Alert mail sent successfully to {}".format(recipientmail))
    except Exception as e:
        print(e)

def start():
    global runOnce
    btn_start.config(state="disabled") # Disable the Start button to prevent multiple camera captures
    while(True):
        Alarm_Status = False
        ret, frame = vid.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        fire = fire_cascade.detectMultiScale(frame, 1.2, 5)

        for (x,y,w,h) in fire:
            cv2.rectangle(frame,(x-20,y-20),(x+w+20,y+h+20),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]

            print("Fire alarm initiated")
            threading.Thread(target=play_alarm_sound_function).start()

            if runOnce == False:
                print("Mail send initiated")
                # Capture image and convert to bytes
                img = cv2.imencode('.jpg', frame)[1].tostring()
                threading.Thread(target=send_mail_function, args=(img,)).start()
                runOnce = True
            if runOnce == True:
                print("Mail is already sent once")
                runOnce = True

        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    vid.release()
    cv2.destroyAllWindows()
    
    
root = tk.Tk()
root.title("Fire Detection System")
root.geometry("400x300")

btn_start = tk.Button(root, text="Start", command=start)
btn_start.pack(pady=10)

root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\KUNAL\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\KUNAL\AppData\Local\Temp\ipykernel_7628\3805982600.py", line 68, in start
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
cv2.error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

